In [2]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Reshape, Add
from keras.layers import Lambda,Concatenate,Flatten,ConvLSTM2D
from keras.layers import Permute,Conv2D,MaxPooling2D,multiply
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from keras import backend as K
from keras.models import load_model
from keras import activations
import sys,glob,io,random
#if '/scratch/cl2840/360video/' not in sys.path:
#    sys.path.insert(0, '/scratch/cl2840/360video/')
from dataLayer import DataLayer
import cost as costfunc
from config import cfg
from dataIO import clip_xyz
import utility as util
from random import shuffle  
import matplotlib.pyplot as plt
import _pickle as pickle
import numpy as np
import pdb
from keras.layers.wrappers import Bidirectional
#from mycode.saliency_CNN import *
import h5py
from tqdm import tqdm
from collections import OrderedDict
from data_generator_including_saliency import *

Using TensorFlow backend.


In [3]:
# experiment = 1 
batch_size = cfg.batch_size
epochs = 1
latent_dim = 32

fps = cfg.fps
num_encoder_tokens = 3*fps
num_decoder_tokens = 6
max_encoder_seq_length = cfg.running_length
max_decoder_seq_length = cfg.predict_step
num_user = 34 #48 
use_generator = True
select_k_neighbours=False
mixing_saliency=False

target_user_only = False
model_others = False
if model_others:
    others_mlp = False
    others_lstm = False #also given gt mean,var
if not model_others:
    mlp_mixing = True
    conv_mixing = False

## utility layers
flatten_layer = Flatten()
expand_dim_layer = Lambda(lambda x: K.expand_dims(x,1))
expand_dim_layer_1 = Lambda(lambda x: K.expand_dims(x,-1))
Concatenatelayer = Concatenate(axis=1)
Concatenatelayer_1 = Concatenate(axis=-1)
get_dim1_layer = Lambda(lambda x: x[:,0,:])

reduce_sum_layer = Lambda(lambda x: K.sum(x, axis=1))#collapsing user dim
sigmoid = activations.get('sigmoid')
# GLU_layer = Lambda(lambda x: multiply([x[0],sigmoid(x[1])]))
GLU_layer = Lambda(lambda x: multiply([x[1],sigmoid(x[0])]))

In [23]:
### ====================Graph def====================
#2 layer encoder
if cfg.input_mean_var:
    encoder_inputs = Input(shape=(None, 6))


        
        


encoder1 = LSTM(latent_dim, stateful=cfg.stateful_across_batch, return_state=True, return_sequences=True)
encoder1_outputs, state_h_1, state_c_1 = encoder1(encoder_inputs)

encoder2 = LSTM(latent_dim, stateful=cfg.stateful_across_batch, return_state=True, return_sequences=True)
encoder2_outputs, state_h_2, state_c_2 = encoder2(encoder1_outputs)

encoder1_states = [state_h_1, state_c_1]
encoder2_states = [state_h_2, state_c_2]

##2 layer decoder
decoder1_states_inputs = encoder1_states
decoder2_states_inputs = encoder2_states


decoder_inputs = Input(shape=(1, num_decoder_tokens))    
    # decoder_inputs = Input(shape=(1, num_encoder_tokens))    
decoder_lstm1 = LSTM(latent_dim, stateful=cfg.stateful_across_batch, return_sequences=True, return_state=True)
decoder_lstm2 = LSTM(latent_dim, stateful=cfg.stateful_across_batch, return_sequences=True, return_state=True)

decoder_dense = Dense(num_decoder_tokens,activation='tanh')


others_fut_inputs = Input(shape=(max_decoder_seq_length,(num_user-1),6))

mixing = Dense(num_decoder_tokens,activation='tanh')




all_outputs = []
inputs = decoder_inputs
for time_ind in range(max_decoder_seq_length):
    # Run the decoder on one timestep
    # 1-layer fclstm
    # decoder_states, state_h, state_c = decoder_lstm(inputs,initial_state=states)
    # decoder_pred = decoder_dense(decoder_states)
    

    decoder1_outputs, state_decoder1_h, state_decoder1_c = decoder_lstm1(inputs,initial_state=decoder1_states_inputs)
    decoder1_states_inputs = [state_decoder1_h, state_decoder1_c]
    decoder2_outputs, state_decoder2_h, state_decoder2_c = decoder_lstm2(decoder1_outputs,initial_state=decoder2_states_inputs)
    decoder2_states_inputs = [state_decoder2_h, state_decoder2_c]

    
    decoder_pred = decoder_dense(decoder2_outputs)
           
          

    # given others' gt mean and variance
    # directly concat others' mean and variance
    gt_mean_var_oth = util.slice_layer(1,time_ind,time_ind+1)(others_fut_inputs) 
            
    concat_outputs = Concatenatelayer([get_dim1_layer(gt_mean_var_oth),decoder_pred])
 
    ##1 layer or 3 layers mixing
    concat_outputs = Flatten()(concat_outputs)
    outputs = mixing(concat_outputs)                
    outputs = expand_dim_layer(outputs)

    all_outputs.append(outputs)
    inputs = outputs
    # inputs = decoder_pred
    # states = [state_h, state_c]

## Concatenate all predictions
decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)



model = Model([encoder_inputs, others_fut_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='Adam', loss='mean_squared_error',metrics=['accuracy'])






In [ ]:
def data_sanity_check(_video_db_tar,_video_db_future_tar,_video_db_future_input_tar):
    sample_ind = np.random.randint(0,_video_db_future_input_tar.shape[0])
    assert (_video_db_tar[sample_ind,:][-1,:]-_video_db_future_input_tar[sample_ind,:][0,:]).sum()==0
    print(np.abs(_video_db_tar[sample_ind,:][-1,:]-_video_db_future_tar[sample_ind,:][0,:]))
    
def _reshape_others_data(_video_db_oth):
    ## to match Input shape: others_fut_inputs
    _video_db_oth = _video_db_oth.transpose((1,2,0,3))
    _video_db_oth = _video_db_oth.reshape((_video_db_oth.shape[0],_video_db_oth.shape[1],_video_db_oth.shape[2],
                    fps,3))
    return _video_db_oth

def get_segment_index(datadb):
    """segment time is used to get the visual/saliency information"""
    #match in time!!!!
    if cfg.use_saliency:
        segment_index_tar = util.get_time_for_visual(datadb)
        segment_index_tar_future = OrderedDict()
        for key in segment_index_tar.keys():
            segment_index_tar_future[key] = np.array(segment_index_tar[key])+max_encoder_seq_length
    return segment_index_tar,segment_index_tar_future

In [8]:



#### ========================================data============================================================
if use_generator:
    video_data_train = pickle.load(open('shanghai_dataset_xyz_train.p','rb'),encoding='latin1')    
    datadb_train = clip_xyz(video_data_train)
    video_data_test = pickle.load(open('shanghai_dataset_xyz_test.p','rb'),encoding='latin1')    
    datadb_test = clip_xyz(video_data_test)

    #saliency
    if cfg.use_saliency:
        segment_index_tar,segment_index_tar_future = get_segment_index(datadb_train)
        mygenerator = generator_train2(datadb_train,segment_index_tar,segment_index_tar_future,phase='train')

        segment_index_tar_test,segment_index_tar_future_test = get_segment_index(datadb_test)
        mygenerator_val = generator_train2(datadb_test,segment_index_tar_test,segment_index_tar_future_test,phase='val')
    else:       
        # no saliency
        mygenerator = generator_train2(datadb_train,phase='train')
        mygenerator_val = generator_train2(datadb_test,phase='val')

else:
    #### create data for tsinghua dataset (train, test) split on users, format5_tsinghua_by_sec_interp
    # video_data_train = pickle.load(open('/scratch/cl2840/360video/temp/tsinghua_after_bysec_interpolation/tsinghua_train_video_data_over_users.p','rb'))    
    # video_data_train = pickle.load(open('/scratch/cl2840/360video/temp/tsinghua_after_bysec_interpolation/tsinghua_train_video_data_over_video.p','rb'))    
    # video_data_train = clip_xyz(video_data_train)
    # datadb = video_data_train.copy()
    # video_data_test = pickle.load(open('/scratch/cl2840/360video/temp/tsinghua_after_bysec_interpolation/tsinghua_test_video_data_over_video.p','rb'))    
    # video_data_test = clip_xyz(video_data_test)
    # datadb = video_data_test.copy()
    # _video_db_tar, _video_db_future_tar, _video_db_future_input_tar, \
    # _video_db_oth,_video_db_future_oth,_video_db_future_input_oth = util.get_data(datadb,pick_user=True,num_user=datadb[0]['x'].shape[0])

    ##### create data format 3or4--4
    # video_data_train = pickle.load(open('/scratch/cl2840/360video/data/shanghai_dataset_xyz_train.p','rb'))    
    # video_data_train = clip_xyz(video_data_train)
    # datadb = video_data_train.copy()
    # video_data_test = pickle.load(open('/scratch/cl2840/360video/data/shanghai_dataset_xyz_test.p','rb'))    
    # video_data_test = clip_xyz(video_data_test)
    # datadb = video_data_test.copy()
    # _video_db_tar, _video_db_future_tar, _video_db_future_input_tar, \
    # _video_db_oth,_video_db_future_oth,_video_db_future_input_oth = util.get_data(datadb,pick_user=True,num_user=34)

    # #### load cached data
    # dataformat = 'formtat2'
    # _video_db_tar = pickle.load(open('./cache/'+dataformat+'/_video_db_tar_exp'+str(experiment)+'.p','rb'))
    # _video_db_future_tar = pickle.load(open('./cache/'+dataformat+'/_video_db_future_tar_exp'+str(experiment)+'.p','rb'))
    # _video_db_future_input_tar = pickle.load(open('./cache/'+dataformat+'/_video_db_future_input_tar_exp'+str(experiment)+'.p','rb'))
    # _video_db_oth = pickle.load(open('./cache/'+dataformat+'/_video_db_oth_exp'+str(experiment)+'.p','rb'))
    # _video_db_future_oth = pickle.load(open('./cache/'+dataformat+'/_video_db_future_oth_exp'+str(experiment)+'.p','rb'))
    # _video_db_future_input_oth = pickle.load(open('./cache/'+dataformat+'/_video_db_future_input_oth_exp'+str(experiment)+'.p','rb'))
    #### load cached data format4 
    # dataformat = 'format4' #shanghaitech
    # option='stride10_cut_head/'
    dataformat = 'format5_tsinghua_by_sec_interp' #tsinghua
    option=''
    _video_db_tar = util.load_h5('./cache/'+dataformat+'/train/'+option+'_video_db_tar.h5','_video_db_tar')
    _video_db_future_tar = util.load_h5('./cache/'+dataformat+'/train/'+option+'_video_db_future_tar.h5','_video_db_future_tar')
    _video_db_future_input_tar = util.load_h5('./cache/'+dataformat+'/train/'+option+'_video_db_future_input_tar.h5','_video_db_future_input_tar')
    _video_db_oth = util.load_h5('./cache/'+dataformat+'/train/'+option+'_video_db_oth.h5','_video_db_oth')
    _video_db_future_oth = util.load_h5('./cache/'+dataformat+'/train/'+option+'_video_db_future_oth.h5','_video_db_future_oth')
    _video_db_future_input_oth = util.load_h5('./cache/'+dataformat+'/train/'+option+'_video_db_future_input_oth.h5','_video_db_future_input_oth')


    _video_db_oth = _reshape_others_data(_video_db_oth)
    _video_db_future_oth = _reshape_others_data(_video_db_future_oth)
    _video_db_tar = _video_db_tar.reshape((_video_db_tar.shape[0],_video_db_tar.shape[1],fps,3))
    _video_db_future_tar = _video_db_future_tar.reshape((_video_db_tar.shape[0],_video_db_tar.shape[1],fps,3))
    _video_db_future_input_tar = _video_db_future_input_tar.reshape((_video_db_tar.shape[0],_video_db_tar.shape[1],fps,3))


    total_num_samples = _video_db_tar.shape[0]
    # num_testing_sample = int(0.15*total_num_samples)
    num_testing_sample = 1

    print('other data shape: ',_video_db_oth.shape)
    print('other data shape: ',_video_db_future_oth.shape)
    print('target user data shape: ',_video_db_tar.shape)
    print('target user data shape: ',_video_db_future_tar.shape)


    if cfg.shuffle_data:
        #### shuffle the whole dataset
        # index_shuf = util.get_shuffle_index(total_num_samples)
        index_shuf = pickle.load(open('index_shuf'+'_exp'+str(experiment)+'.p','rb'))
        print('Shuffle data before training and testing.')
        _video_db_tar = util.shuffle_data(index_shuf,_video_db_tar)
        _video_db_future_tar = util.shuffle_data(index_shuf,_video_db_future_tar)
        _video_db_future_input_tar = util.shuffle_data(index_shuf,_video_db_future_input_tar)

        # _video_db_oth = util.shuffle_data(index_shuf,_video_db_oth)
        _video_db_future_oth = util.shuffle_data(index_shuf,_video_db_future_oth)
        # _video_db_future_input_oth = util.shuffle_data(index_shuf,_video_db_future_input_oth)


    #### prepare training data
    if cfg.input_mean_var:
        ### target user
        encoder_input_data = util.get_gt_target_xyz(_video_db_tar[:-num_testing_sample,:,:])
        decoder_target_data = util.get_gt_target_xyz(_video_db_future_tar[:-num_testing_sample,:,:])
        # encoder_input_data = util.get_gt_target_xyz(_video_db_oth_all[:-num_testing_sample,:,:])
        # decoder_target_data = util.get_gt_target_xyz(_video_db_future_oth_all[:-num_testing_sample,:,:])
        # ### other users
        others_fut_input_data = util.get_gt_target_xyz_oth(_video_db_future_oth[:-num_testing_sample])
        if not cfg.teacher_forcing:
            decoder_input_data = encoder_input_data[:,-1,:][:,np.newaxis,:]
        else:
            decoder_input_data = util.get_gt_target_xyz(_video_db_future_input_tar[:-num_testing_sample,:,:])
            # decoder_input_data = util.get_gt_target_xyz(_video_db_future_input_oth_all[:-num_testing_sample,:,:])

    else:
        ### target user
        _video_db_tar = _video_db_tar.reshape((_video_db_tar.shape[0],_video_db_tar.shape[1],-1))
        encoder_input_data = _video_db_tar[:-num_testing_sample,:,:]
        # decoder_target_data = _video_db_future_tar[:-num_testing_sample,:,:][:,:,np.newaxis,:,:]#predict raw
        decoder_target_data = util.get_gt_target_xyz(_video_db_future_tar[:-num_testing_sample,:,:])#predict mean/var
        # decoder_input_data = _video_db_tar[:-num_testing_sample,-1,:][:,np.newaxis,np.newaxis,:]
        decoder_input_data = util.get_gt_target_xyz(encoder_input_data[:,-1,:][:,np.newaxis,:])
        ### other users
        # others_fut_input_data = _video_db_future_oth[:-num_testing_sample][:,:,np.newaxis,:]
        others_fut_input_data = util.get_gt_target_xyz_oth(_video_db_future_oth[:-num_testing_sample])


In [ ]:


### ====================Training====================

tag='mlpmixing_softmaxgating_sep14'

model_checkpoint = ModelCheckpoint(tag+'{epoch:02d}-{val_loss:.4f}.h5', monitor='val_loss', save_best_only=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                 patience=10, min_lr=1e-6)
stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')


num_samples = 10000

tqdm(model.fit_generator(mygenerator,steps_per_epoch=num_samples, epochs=epochs,
                validation_data=mygenerator_val, validation_steps=100,
                callbacks=[model_checkpoint, reduce_lr, stopping],
                initial_epoch=0))


In [9]:
### ====================Testing===================

video_data_test = pickle.load(open('shanghai_dataset_xyz_test.p','rb'),encoding='latin1')    
datadb_test = clip_xyz(video_data_test)


mygenerator_test = generator_train2(datadb_test,phase='test')

test_out = []
gt_out = []
while len(test_out)<1261:
    #x,gt_temp = mygenerator_test.next()
    x,gt_temp = next(mygenerator_test)
    test_out_temp = model.predict_on_batch(x)
    test_out.append(test_out_temp)
    gt_out.append(gt_temp)

pickle.dump(test_out,open('decoded_sentence'+tag+'.p','wb'))
pickle.dump(gt_out,open('gt_sentence_list'+tag+'.p','wb'))
print('Testing finished!')

Testing finished!


In [128]:
### ====================Data Reshaping===================

temp_test_out = pickle.load(open('C:/Users/yaobo/Desktop/project/decoded_sentencemlpmixing_softmaxgating_sep14'+'.p','rb'))
temp_gt_out = pickle.load(open('C:/Users/yaobo/Desktop/project/gt_sentence_listmlpmixing_softmaxgating_sep14'+'.p','rb'))

test_out=np.concatenate(temp_test_out)
gt_out=np.concatenate(temp_gt_out)
    

pickle.dump(test_out,open('decoded_sentence'+'.p','wb'))
pickle.dump(gt_out,open('gt_sentence_list'+'.p','wb'))




